---

## Dense Model v1

---

Multivariate usage of a Dense NN.

In [ ]:
debug = True
DRIVE_PATH = "/content/drive/MyDrive/data606"

# Set the location of this script in GDrive
SCRIPT_PATH = DRIVE_PATH + "/src/"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd $SCRIPT_PATH

/content/drive/MyDrive/data606/src


In [ ]:
# Load model classes
%run -i "./Model_Base.ipynb"

In [ ]:
from datetime import datetime as dt
from keras.layers import Dense,MaxPooling1D
from keras.layers import Flatten,RepeatVector
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, TensorBoard, Callback, EarlyStopping

In [ ]:
class Model_Densev1(Base_Model):

  def get_name(self):
    return "Densev1"

  def train(self, X_train=None, y_train=None, num_features=None, dataset=None):
    """
    Build and train the model.
    """
    early_stop = EarlyStopping(monitor = "loss", mode = "min", patience = 25)
    model = Sequential()
    model.add(MaxPooling1D(pool_size=2, input_shape=(self.WINDOW_SIZE, num_features)))
    model.add(Dense(100, activation='relu', kernel_initializer='glorot_uniform'))
    model.add(Dense(100, activation='relu', kernel_initializer='glorot_uniform'))
    model.add(Dense(100, activation='relu', kernel_initializer='glorot_uniform'))
    model.add(Flatten())
    model.add(Dense(self.NUM_LABELS*self.LABEL_WINDOW, kernel_initializer='glorot_uniform'))
    if (self.LABEL_WINDOW > 1):
      # reshape as => [batch, out_steps, labels]
      model.add(Reshape([self.LABEL_WINDOW, self.NUM_LABELS]))

    model.compile(loss='mae', optimizer='adam')

    if (dataset is not None):
      self.model_hist = model.fit(dataset, epochs=self.NUM_EPOCHS, callbacks = [early_stop], verbose=(1 if self.debug else 0))
    else:
      self.model_hist = model.fit(X_train, y_train, epochs=self.NUM_EPOCHS, verbose=1, callbacks = [early_stop] )
    self.model = model
    return self.model_hist

